# プロスペクト理論
関数のパラメータを同定する（求める）のに，カーブフィッティングを用いる。<br>
scipy.optimize.curve_fit<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html<br>

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.optimize as opt

import matplotlib.pyplot as plt
%matplotlib inline

#### 確率加重関数（probability weighting function）
$$
w(p) = \frac{{{p^\gamma }}}{{{{\left\{ {{p^\gamma } + {{\left( {1 - p} \right)}^\gamma }} \right\}}^{1/\gamma }}}}
$$

In [ ]:
def pwfun(p, g):
    return p**g/pow((p**g+(1-p)**g), 1/g)

In [ ]:
p = np.linspace(0.0, 1.0, 100)
w1 = pwfun(p, 1)
w2 = pwfun(p, 2)
w09 = pwfun(p, 0.9)
w08 = pwfun(p, 0.8)
w07 = pwfun(p, 0.7)
w06 = pwfun(p, 0.6)
w05 = pwfun(p, 0.5)
w03 = pwfun(p, 0.3)
w02 = pwfun(p, 0.2)


fig = plt.subplots(figsize=(5,5))
plt.plot(p, w1, label="g=1", color='k')
plt.plot(p, w2, label="g=2")
plt.plot(p, w07, label="g=0.7")
plt.plot(p, w05, label="g=0.5")
plt.plot(p, w03, label="g=0.3")
#plt.plot(p, w02, label="g=0.2")
plt.legend()
plt.grid()
plt.xlabel('p')
plt.ylabel('w(p)')
#plt.savefig('fig_DE_Prospect01.pdf', bbox_inches='tight')

#### 期待効用の計算
問題１：u1a, u1bは，それぞれ，くじ1-a，くじ1-bに対する期待効用

In [ ]:
gamma = 0.5
u1a = pwfun(1.0, gamma)*(1.0*(10**5))
u1b = pwfun(0.2, gamma)*(1.0*(10**5)) + pwfun(0.79, gamma)*(1.2*(10**5))
print('u1a=',u1a, '   u1b=',u1b)

問題２：u2a, u2bは，それぞれ，くじ2-a，くじ2-bに対する期待効用

In [ ]:
u2a = pwfun(0.11, gamma)*(10**5)
u2b = pwfun(0.1, gamma)*(2.5*10**5)
print('u2a=',u2a, '   u2b=',u2b)

## パラメータ同定
パラメータ推定ともいう。論文 Tversky, A., & Kahneman, D. (1992). Advances in prospect theory: Cumulative representation of uncertainty. Journal of Risk and Uncertainty, 5, 297-323.
のデータを下記のように用いている。

In [ ]:
prob_a = np.array([0,  1,  2,5,10,20,50,80,90,95,98,99,100])
prob_s = np.array([0,5.5,8.1,13.2,18.6,26.1,42.1,60.1,71.2,79.3,87.1,91.2,100])

In [ ]:
print(len(prob_a),len(prob_s))

In [ ]:
prob_an = prob_a/100
prob_sn = prob_s/100
p = np.linspace(0.0, 1.0, 100)

#### データのプロット
matplotlib.markers: https://matplotlib.org/stable/api/markers_api.html

In [ ]:
fig = plt.subplots(figsize=(4,4))
plt.scatter(prob_an, prob_sn)
plt.plot(p, p, color='k')
plt.grid()
plt.xlabel('p')
plt.ylabel('w(p)')
#plt.savefig('fig_DE_Prospect02.pdf', bbox_inches='tight')

#### パラメータ同定

In [ ]:
param, cov = opt.curve_fit(pwfun, prob_an, prob_sn, p0=[0.5,])

In [ ]:
param

In [ ]:
param[0]

#### 同定した関数のプロット

In [ ]:
wo = pwfun(p, param[0])

fig = plt.subplots(figsize=(5,5))
plt.plot(p,wo)
plt.plot(p,w1, color='k')
plt.scatter(prob_an, prob_sn)
plt.grid()
plt.xlabel('p')
plt.ylabel('w(p)')
#plt.savefig('fig_DE_Prospect_CurveFit01.pdf', bbox_inches='tight')

#### 期待効用の計算

In [ ]:
gamma = param[0]
u1a = pwfun(1.0, gamma)*(1.0*(10**5))
u1b = pwfun(0.2, gamma)*(1.0*(10**5)) + pwfun(0.79, gamma)*(1.2*(10**5))
print('u1a=',u1a, '   u1b=',u1b)

u2a = pwfun(0.11, gamma)*(10**5)
u2b = pwfun(0.1, gamma)*(2.5*10**5)
print('u2a=',u2a, '   u2b=',u2b)

## 3次関数のカーブフィッティング

In [ ]:
def funCube(p, a, b, c, d):
    return a*(p**3)+b*(p**2)+c*p+d

param2, cov = opt.curve_fit(funCube, prob_an, prob_sn)
print(param2)

In [ ]:
y = param2[0]*(p**3)+param2[1]*(p**2)+param2[2]*p+param2[3]

fig = plt.subplots(figsize=(5,5))
plt.plot(p, y)
plt.plot(p, w1, color='k')
plt.scatter(prob_an, prob_sn)
plt.grid()
plt.xlabel('p')
plt.ylabel('w(p)')
#plt.savefig('fig_DE_Prospect_CurveFit02.pdf', bbox_inches='tight')

## 別の確率加重関数
$$
w(p) = \exp \left\{ { - \alpha {{\left( { - {{\log }_e}p} \right)}^\gamma }} \right\}
$$

In [ ]:
def funExp(p, g, alp):
    return np.exp( -alp*((-np.log(p))**g))

In [ ]:
alp = 0.8
v1 = funExp(p, 1, alp)
v2 = funExp(p, 2, alp)
v09 = funExp(p, 0.9, alp)
v08 = funExp(p, 0.8, alp)
v07 = funExp(p, 0.7, alp)
v06 = funExp(p, 0.6, alp)
v05 = funExp(p, 0.5, alp)
v03 = funExp(p, 0.3, alp)

fig = plt.subplots(figsize=(5,5))
plt.plot(p, p, color='k')
plt.plot(p, v1, label="g=1")
plt.plot(p, v2, label="g=2")
plt.plot(p, v07, label="g=0.7")
plt.plot(p, v05, label="g=0.5")
plt.plot(p, v03, label="g=0.3")
plt.legend()
plt.grid()
plt.xlabel('p')
plt.ylabel('w(p)')